In [1]:
%%capture
!pip install transformers datasets
!python -m pip install huggingface_hub


In [2]:
MINE = "jimregan/wav2vec2-swedish-riksdag-100h"
KBLAB = "KBLab/wav2vec2-large-voxrex-swedish"


In [3]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from datasets import load_dataset

cv6 = load_dataset("mozilla-foundation/common_voice_6_1", "sv-SE", split="test", use_auth_token=True)
cv11 = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating other split: 0 examples [00:00, ? examples/s]

Generating invalidated split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]





Reading metadata...: 5699it [00:00, 59662.89it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]






Reading metadata...: 1346it [00:00, 51053.37it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/sv-SE/11.0.0/2c65b95d99ca879b1b1074ea197b65e0497848fd697fdb0582e0f6b75b6f4da0. Subsequent calls will reuse this data.


In [6]:
from datasets import load_dataset, load_metric, Audio

cv6 = cv6.cast_column("audio", Audio(sampling_rate=16_000))
cv11 = cv11.cast_column("audio", Audio(sampling_rate=16_000))
cv6 = cv6.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
cv11 = cv11.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.1 MB/s eta 0:00:00


In [8]:
wer_metric = load_metric("wer")

<ipython-input-8-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [9]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

mymodel = AutoModelForCTC.from_pretrained(MINE)
myprocessor = Wav2Vec2Processor.from_pretrained(MINE)
kbmodel = AutoModelForCTC.from_pretrained(KBLAB)
kbprocessor = Wav2Vec2Processor.from_pretrained(KBLAB)


/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import torch
import torchaudio
import re

def speech_file_to_array_fn(batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    resampler = torchaudio.transforms.Resample(48_000, 16_000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [12]:
cv6 = cv6.map(speech_file_to_array_fn)
#cv11 = cv11.map(speech_file_to_array_fn)

Map:   0%|          | 0/2027 [00:00<?, ? examples/s]

In [14]:
def evaluate(batch):
    inputs = myprocessor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
    mymodel.to("cuda")
    with torch.no_grad():
        logits = mymodel(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits    
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = myprocessor.batch_decode(pred_ids)
    return batch
result = cv6.map(evaluate, batched=True, batch_size=8)

Map:   0%|          | 0/2027 [00:00<?, ? examples/s]

In [16]:
print("WER: {:2f}".format(100 * wer_metric.compute(predictions=result["pred_strings"], references=result["sentence"])))

WER: 20.175122


In [17]:
def kbevaluate(batch):
    inputs = kbprocessor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
    kbmodel.to("cuda")
    with torch.no_grad():
        logits = kbmodel(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits    
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = kbprocessor.batch_decode(pred_ids)
    return batch
result = cv6.map(kbevaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer_metric.compute(predictions=result["pred_strings"], references=result["sentence"])))

Map:   0%|          | 0/2027 [00:00<?, ? examples/s]

WER: 9.053981
